# Set up the Edgar Postgres DB

### Set up imports

In [1]:
import os
import psycopg2

### Set up database connection

Make sure the correct variables exist in your environment.

In [2]:
# Load credentials from environment. 
POSTGRES_ADDRESS = os.environ['POSTGRES_ADDRESS']
POSTGRES_PORT = os.environ['POSTGRES_PORT']
POSTGRES_USERNAME = os.environ['POSTGRES_USERNAME']
POSTGRES_PASSWORD = os.environ['POSTGRES_PASSWORD']
POSTGRES_PASSWORD = "gZ3uoUyCtB8"
POSTGRES_DBNAME = os.environ['POSTGRES_DBNAME']

In [3]:
# Create connection and cursor    
conn = psycopg2.connect(host=POSTGRES_ADDRESS,
                  database=POSTGRES_DBNAME,
                  user=POSTGRES_USERNAME,
                  password=POSTGRES_PASSWORD,
                  port=POSTGRES_PORT)
cur = conn.cursor()

### Create the profile, income, and balance sheet tables

(Only run this if it doesn't already exist)

In [4]:
# Create the Profile table
cur.execute("""CREATE TABLE profile_table
               (symbol varchar(5),
                price float,
                beta float,
                volAvg bigint,
                mktCap bigint,
                lastDiv float,
                changes float,
                companyName varchar(100),
                exchangeShortName varchar(10),
                industry varchar(100),
                website varchar(100),
                description text,
                ceo varchar(100),
                sector varchar(100),
                country varchar(20),
                fullTimeEmployees integer,
                phone varchar(20),
                address text,
                city varchar(50),
                state varchar(20),
                dcf float,
                ipoDate timestamp);""") 

# Commit the table creation transaction.
conn.commit()

In [5]:
# Create the Income Statement table
cur.execute("""CREATE TABLE income_table
                (symbol varchar(5),
                date timestamp,
                period varchar(5),
                revenue bigint, 
                costOfRevenue bigint, 
                grossProfit bigint, 
                grossProfitRatio float, 
                researchAndDevelopmentExpenses bigint, 
                generalAndAdministrativeExpenses bigint, 
                sellingAndMarketingExpenses bigint,
                otherExpenses bigint, 
                operatingExpenses bigint, 
                costAndExpenses bigint, 
                interestExpense bigint, 
                depreciationAndAmortization bigint, 
                ebitda bigint, 
                ebitdaratio float, 
                operatingIncome bigint, 
                operatingIncomeRatio float,
                totalOtherIncomeExpensesNet bigint, 
                incomeBeforeTax bigint, 
                incomeBeforeTaxRatio float, 
                incomeTaxExpense bigint, 
                netIncome bigint, 
                netIncomeRatio float, 
                eps float, 
                epsdiluted float, 
                weightedAverageShsOut bigint, 
                weightedAverageShsOutDil bigint);""") 
    
# Commit the table creation transaction.
conn.commit()

In [7]:
# Create the Balance Sheet Statement table
cur.execute("""CREATE TABLE balance
                (symbol varchar(5),
                date timestamp,
                period varchar(5),
                cashAndCashEquivalents bigint,
                shortTermInvestments bigint,
                cashAndShortTermInvestments bigint,
                netReceivables bigint,
                inventory bigint,
                totalCurrentAssets bigint,
                propertyPlantEquipmentNet bigint,
                goodwill bigint,
                intangibleAssets bigint,
                longTermInvestments bigint,
                taxAssets bigint,
                totalNonCurrentAssets bigint,
                otherAssets bigint,
                totalAssets bigint,
                accountPayables bigint,
                shortTermDebt bigint,
                taxPayables bigint,
                deferredRevenue bigint,
                totalCurrentLiabilities bigint,
                longTermDebt bigint,
                deferredRevenueNonCurrent bigint,
                totalLiabilities bigint,
                commonStock bigint,
                retainedEarnings bigint,
                totalStockholdersEquity bigint,
                totalLiabilitiesAndStockholdersEquity bigint,
                totalInvestments bigint,
                totalDebt bigint,
                netDebt bigint);""") 

# Commit the table creation transaction.
conn.commit()

### Display contents of the income table

Feel free to run this interactively to debug table population.

In [59]:
conn.commit()
query = """SELECT * FROM income_table WHERE symbol = 'GOOGL';"""
cur.execute(query)
cur.fetchall()

[('GOOGL',
  datetime.datetime(2020, 12, 31, 0, 0),
  'FY',
  182527000000,
  84732000000,
  97795000000,
  0.5357837470620785,
  27573000000,
  11052000000,
  17946000000,
  -1246000000,
  56571000000,
  141303000000,
  135000000,
  13697000000,
  54921000000,
  0.3008924707029645,
  41224000000,
  0.22585151785763202,
  5128000000,
  48082000000,
  0.2634240413747007,
  7813000000,
  40269000000,
  0.22061941520980458,
  57.64605844055451,
  57.64605844055451,
  698556000,
  698556000),
 ('GOOGL',
  datetime.datetime(2019, 12, 31, 0, 0),
  'FY',
  161857000000,
  71896000000,
  89961000000,
  0.5558054331910266,
  26018000000,
  9551000000,
  18464000000,
  5394000000,
  54033000000,
  127626000000,
  100000000,
  11781000000,
  46012000000,
  0.28427562601555695,
  34231000000,
  0.2114891540063142,
  -79000000,
  39625000000,
  0.24481486744471972,
  5282000000,
  34343000000,
  0.2121811228430034,
  49.16284,
  49.16284,
  698556000,
  698556000),
 ('GOOGL',
  datetime.datetime(20

In [55]:
conn.commit()
query = """
DELETE FROM profile_table WHERE symbol='BRK-B';
"""
cur.execute(query)
conn.commit()

In [20]:
query = """
DELETE FROM profile_table;
"""
cur.execute(query)
query = """
DELETE FROM income_table;
"""
cur.execute(query)
query = """
DELETE FROM balance;
"""
cur.execute(query)
conn.commit()

In [30]:
# Commit the transaction.
conn.commit()

In [36]:
# To delete all rows, run """DELETE FROM income;"""
# To delete the table, run """DROP TABLE income;"""